In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import yfinance as yf
from datetime import datetime, timedelta

import statsmodels.api as sm
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose

from arch import arch_model

from pmdarima.arima import auto_arima

from statsmodels.tsa.arima.model import ARIMA
from scipy.stats.distributions import chi2

import plotly.express as px
import plotly.graph_objects as go

In [2]:
import scipy.stats
import pylab

In [3]:
n_years = 10
years_in_days = n_years * 365

start_date = (datetime.today() - timedelta(days=years_in_days)).strftime("%Y-%m-%d")

In [4]:
index_symbols = ['SPY', 'QQQ', 'VUG', 'IWF', 'XLF', 'IJR']

data = yf.download(index_symbols, start_date)

Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\filip\Anaconda3\envs\TFc\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\filip\Anaconda3\envs\TFc\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\filip\Anaconda3\envs\TFc\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Users\filip\Anaconda3\envs\TFc\lib\site-packages\yfinance\multi.py", line 170, in _download_one_threaded
    actions, period, interval, prepost, proxy, rounding)
  File "C:\Users\filip\Anaconda3\envs\TFc\lib\site-packages\yfinance\multi.py", line 181, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "C:\Users\filip\Anaconda3\envs\TFc\lib\site-packages\yfinance\base.py", line 65, in __init__
    "yearly": utils.empty_df(),
  File "C:\Users\filip\Anaconda3\envs\TFc\lib\site-packages\yfinance\utils.

KeyboardInterrupt: 

In [ ]:
adj_close = data['Adj Close']

In [ ]:
adj_close.describe()

In [ ]:
d_rets = adj_close.pct_change()

In [ ]:
cum_d_rets = d_rets.apply(lambda x: x.add(1, fill_value=0).cumprod() - 1)

In [ ]:

px.line(data_frame=cum_d_rets, title='Cumulative returns (%)')

In [ ]:
#d_rets.apply(lambda x: x.add(100, fill_value=0).cumprod()).plot()

px.line(d_rets.apply(lambda x: x.add(1, fill_value=0).cumprod() * 100))

In [ ]:
def plot_c_rets(rets_df, chart_title):
    
    def make_line(rets_df, column, alt_name=None):
        data = rets_df[[column]]
        name = column
        if alt_name is not None:
            name = f"{alt_name} ({column})"

        return go.Scatter(x=data.index, y=data[column], name=name)


    def make_chart(rets_df, emphasize=None):
        alt_names = {}
        data = []
        for column in rets_df:
            alt_name = None
            if column in alt_names:
                alt_name = alt_names[column]
            chart = make_line(rets_df, column, alt_name)

            if emphasize is not None:
                if type(emphasize) != list:
                    emphasize = [emphasize]
                if column not in emphasize:
                    chart.line.width = 1
                    chart.mode = 'lines'
                else:
                    chart.line.width = 3
                    chart.mode = 'lines+markers'
            data.append(chart)
        return data
    
    
    data = make_chart(rets_df)

    layout = {'template': 'plotly_dark',
              'title': chart_title,
              'xaxis': {'title': {'text': 'Date'}},
              'yaxis': {'title': {'text': 'Cumulative Total Return'},
                        'tickformat': '.0%'}}

    return go.Figure(data=data, layout=layout)
    

In [ ]:
plot_c_rets(cum_d_rets, 'returns')

In [ ]:
## qq plot 
scipy.stats.probplot(adj_close['SPY'], plot=pylab)
pylab.show()

In [ ]:
### split data

In [ ]:
# 80 - 20
size = int(len(d_rets)*0.8)

In [ ]:
train_data = d_rets.iloc[:size]
test_data = d_rets.iloc[size:] 

In [ ]:
train_data.tail()

In [ ]:
test_data.head()

In [ ]:
### white noise.. 

In [ ]:
### random walk...

In [ ]:
### stationarity # covariance stationarity (weak form) (constat variance - mean)

In [ ]:
## Dickey-Fuller (DF) test (stationarity weak form)

# test statistic (compare it to critical values (1%, 5%, 10%) (if < -> sta.))
# p-value (% chance of not rejecting the null)
# n. lags (t-stat)
# n. of obs
# (autocorr) (lower -> easier predictions)

# rets DF
sts.adfuller(d_rets['SPY'].dropna())

In [ ]:
# price DF
sts.adfuller(adj_close['SPY'].dropna())

In [ ]:
### SEASONALITY
# Decompose (trend-seasonal-residual) (pattern-cyclical effects-prediction error)

# Naive decomposition (linear rel.) (additive - multiplicative)
# add.: observed = trend + seasonal + residual 
# mul.: observed = trend * seasonal * residual

tseries_ = adj_close['QQQ'].asfreq(freq='b', method='ffill')

s_dec_add = seasonal_decompose(tseries_, model="additive")
s_dec_mul = seasonal_decompose(tseries_, model="multiplicative")

In [ ]:
s_dec_add.plot()  # no concrete cyclical pattern with naive 
plt.show()

In [ ]:
s_dec_mul.plot() # no concrete cyclical pattern with naive 
plt.show()

In [ ]:
res = seasonal_decompose(tseries_, model='additive')

fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(15,8))
res.trend.plot(ax=ax1)
res.resid.plot(ax=ax2)
res.seasonal.plot(ax=ax3)

In [ ]:
### CORRELATION (PAST-PRESENT values)
### AUTOCORRELATION (lagged values)
### ACF - PACF

In [ ]:
# ACF
sgt.plot_acf(tseries_, lags=40, zero=False)
plt.title('QQQ ACF')
plt.show()

In [ ]:
sgt.plot_acf(d_rets['QQQ'].dropna(), lags=40, zero=False)
plt.show()

In [ ]:
# PACF (direct effect)

sgt.plot_pacf(tseries_, lags=40, zero=False, method=('ols'))
plt.title('QQQ PACF')
plt.show()

In [ ]:
sgt.plot_pacf(d_rets['QQQ'].dropna(), lags=40, zero=False, method=('ols'))
plt.show()

In [ ]:
### select correct model (significance-parsimonious-no LLR (number of lags) (AIC-BIC)-residual=white noise)

In [ ]:
### AUTOREGRESSIVE MODELS  (AR MA ARMA ARIMA ARCH GARCH)

In [ ]:
# AR (linear) (AR(1)) (xt = C + b Xt-1 + et)(|b|<1)

In [ ]:
# ACF PACF for optimal n. of lags 

In [ ]:
## QQQ
qqq_p = adj_close['QQQ'].asfreq('b', 'ffill')
sgt.plot_acf(qqq_p, lags=40, zero=False)
plt.title('QQQ price ACF')
plt.show()

In [ ]:
sgt.plot_pacf(qqq_p, lags=40, alpha=0.05, zero=False, method=('ols'))
plt.title('QQQ price PACF')
plt.show()  

In [ ]:
# AR(1)

ar_model = ARIMA(qqq_p, order=(1,0,0))

In [ ]:
ar_results = ar_model.fit()

In [ ]:
ar_results.summary()

In [ ]:
ar2_model = ARIMA(qqq_p, order=(2,0,0))
ar2_results = ar2_model.fit()
ar2_results.summary()

In [ ]:
ar3_model = ARIMA(qqq_p, order=(3,0,0))
ar3_results = ar3_model.fit()
ar3_results.summary()

In [ ]:
# higher log likelihood lowe info criteria

In [ ]:
# LLR TEST
def LLR_test(mod1, mod2, DF=1):
    L1 = mod1.fit().llf
    L2 = mod2.fit().llf
    LR = (2*(L2-L1))
    p = chi2.sf(LR, DF).round(3)
    return p

In [ ]:
LLR_test(ar2_model, ar3_model)

In [ ]:
### using returns - stationary 

In [ ]:
qqq_r = d_rets['QQQ'].asfreq('b', 'ffill')[1:]

In [ ]:
# DF test
sts.adfuller(qqq_r)

In [ ]:
sgt.plot_acf(qqq_r, lags=40, zero=False)
plt.title('QQQ rets ACF')
plt.show()

In [ ]:
sgt.plot_pacf(qqq_p, lags=40, alpha=0.05, zero=False, method=('ols'))
plt.title('QQQ rets PACF')
plt.show()  

In [ ]:
# AR(1) for rets

ar1r_model = ARIMA(d_rets['QQQ'].asfreq('b'), order=(1,0,0))
ar1r_results = ar1r_model.fit()
ar1r_results.summary()

In [ ]:
ar2r_model = ARIMA(qqq_r, order=(2,0,0))
ar2r_results = ar2r_model.fit()
ar2r_results.summary()

In [ ]:
LLR_test(ar1r_model, ar2r_model)

In [ ]:
# norm values 

cum_d_rets.plot()

In [ ]:
d_retsn = d_rets.dropna()
rets_n = d_retsn.div(d_retsn.iloc[0]).mul(100) #.plot()

In [ ]:
ar1rn_model = ARIMA(rets_n['QQQ'].asfreq('b'), order=(1,0,0))
ar1rn_results = ar1rn_model.fit()
ar1rn_results.summary()

In [ ]:
# residuals

res = ar1r_results.resid

In [ ]:
res.mean()

In [ ]:
res.var()

In [ ]:
sts.adfuller(res.dropna())

In [ ]:
sgt.plot_acf(res.dropna(), zero=False, lags=40)
plt.show()

In [ ]:
res.plot(figsize=(10,5))

In [ ]:
## MA rt = c + b et-1 + et

In [ ]:
# rets ACF

sgt.plot_acf(d_rets['QQQ'][1:], zero=False, lags=40)
plt.title('QQQ rets ACF')
plt.show()

In [ ]:
# MA(1)


rets_qqq = d_rets['QQQ'][1:].asfreq('b')
# empty 

In [ ]:
model_ma1_rets = ARIMA(rets_qqq, order=(0,0,1))
results_ma1_rets = model_ma1_rets.fit()
results_ma1_rets.summary()

In [ ]:
#MA(2)
model_ma2_rets = ARIMA(rets_qqq, order=(0,0,2))
results_ma2_rets = model_ma2_rets.fit()
results_ma2_rets.summary()

In [ ]:
model_ma3_rets = ARIMA(rets_qqq, order=(0,0,3))
results_ma3_rets = model_ma3_rets.fit()
results_ma3_rets.summary()

In [ ]:
## LLR test (df>1)
LLR_test(model_ma1_rets, model_ma3_rets, DF=2)

In [ ]:
# res.

rets_res = results_ma3_rets.resid[1:]

print(rets_res.mean())
print(rets_res.var())
print(np.sqrt(rets_res.var()))

In [ ]:
rets_res.plot()

In [ ]:
sts.adfuller(rets_res.dropna())

In [ ]:
# normalized returns
norm_qqq = rets_qqq.div(rets_qqq.iloc[0]).mul(100)

In [ ]:
sgt.plot_acf(norm_qqq.dropna(), zero=False, lags=40)
plt.title('QQQ rets ACF')
plt.show()

In [ ]:
## ARMA (1,1)  #AIC <

In [ ]:
model_arma1_rets = ARIMA(rets_qqq, order=(1,0,1))
results_arma1_rets = model_arma1_rets.fit()
results_arma1_rets.summary()

In [ ]:
# ARIMA (non-stationary data) (returns)

In [ ]:
# ARIMAX rt = c + bX + phi rt-1 + a et-1 + et

spy = d_rets['SPY'][1:].asfreq('b').dropna()
qqq = d_rets['QQQ'][1:].asfreq('b').dropna()

In [ ]:
model_armax1_rets = ARIMA(qqq, exog=spy, order=(1,0,1))
results_armax1_rets = model_armax1_rets.fit()
results_armax1_rets.summary()

In [ ]:
# seasonality SARIMAX

# s=1 (no season)

In [ ]:
model_armax1_rets = ARIMA(qqq, exog=spy, order=(1,0,1), seasonal_order=(2,0,1,5))
results_armax1_rets = model_armax1_rets.fit()
results_armax1_rets.summary()

In [ ]:
### ARCH(q)  condiotional heteroskedasticity var(y/yt-1) = c + a e^2t-1
# rt = mut + et
# PACF 
# (squared rets)
# volatility - egarch

In [ ]:
# squared rets

In [ ]:
qqq.plot()

In [ ]:
qqq_sq = (qqq.mul(qqq))
qqq_sq.plot()

In [ ]:
sgt.plot_pacf(qqq, lags=40, alpha=0.05, zero=False, method=('ols'))
plt.show()

In [ ]:
sgt.plot_pacf(qqq_sq, lags=40, alpha=0.05, zero=False, method=('ols'))
plt.show()

In [ ]:
## ARCH(1)

qqq_r = 100 * adj_close['QQQ'].pct_change().dropna()
spy_r = 100 * adj_close['SPY'].pct_change().dropna()
qqq_rs = qqq_r.mul(qqq_r)

In [ ]:
qqq_r.plot()

In [ ]:
qqq_rs.plot()

In [ ]:
arch1_mod = arch_model(qqq_rs)
results_arch1 = arch1_mod.fit()
results_arch1.summary()

In [ ]:
# df model

In [ ]:
# simple ARCH 
arch1_mod = arch_model(qqq_rs, mean='Constant', vol='ARCH', p=1)
results_arch1 = arch1_mod.fit()
results_arch1.summary()

In [ ]:
arch1_mod = arch_model(qqq_rs, mean='AR', lags=[2,3,6], vol='ARCH', p=1, dist='ged')
results_arch1 = arch1_mod.fit()
results_arch1.summary()

In [ ]:
arch2_mod = arch_model(qqq_rs, mean='Constant', vol='ARCH', p=2)
results_arch2 = arch2_mod.fit()
results_arch2.summary()

In [ ]:
#### GARCH 
# arch: past returns conditional - MA
# garch: past rets + past cond. variances - AR

In [ ]:
## GARCH(1,1)
garch1_mod = arch_model(qqq_r, mean='Constant', vol='GARCH', p=1, q=1)
results_garch1 = garch1_mod.fit(update_freq=2)
results_garch1.summary()

In [ ]:
### An auto empirical model selection (AICs LogLikel) (auto arima)

# (HIgh LLF - low AIC) 
# low BIC

# problems - one criterion # params complexity

In [ ]:
model_auto = auto_arima(qqq_r)

In [ ]:
model_auto #ARMA(2,2)

In [ ]:
model_auto.summary()  # only consider AIC

In [ ]:
# args 

model_auto = auto_arima(qqq_r, exogenous=spy_r.values.reshape(-1,1), max_order=None,
                       max_p=7, max_q=7, max_d=2, max_P=5, max_Q=5, max_D=2, maxiter=50,
                       alpha=0.05, n_jobs=-1, trend='ct', information_criterions='oob', 
                       out_of_sample_size=int(len(qqq_r)*0.2))

In [ ]:
model_auto

In [ ]:
model_auto.summary()